# LeNet - Entrenamiento y Evaluación

Este notebook implementa el entrenamiento de LeNet base y sus variantes siguiendo la metodología IMRA descrita en el README.

## Configuración de Entrenamiento
- **Épocas**: 60
- **Batch size**: 64  
- **Optimizador**: SGD (momentum=0.9, weight_decay=5e-4)
- **Learning rate**: 0.001
- **Criterio**: CrossEntropyLoss
- **Semilla fija**: 42 para reproducibilidad

In [ ]:
import sys
sys.path.append('../')

import torch
import torch.nn as nn
import os
import numpy as np
import random

from models.lenet import CustomLenet
from models.lenet_variants import CustomLenet_NoBN, CustomLenet_NoDropout
from utils.dataset import create_data_loaders, get_dataset_info
from utils.training import train_model, test_model, count_parameters
from utils.visualization import plot_training_curves, plot_confusion_matrix

# Configurar semillas para reproducibilidad
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo: {device}")

## 1. Carga de Datos

In [ ]:
# Configuración de datos
data_dir = '../data'
batch_size = 64
input_size = 224

# Obtener información del dataset
dataset_info = get_dataset_info(data_dir)
print("Información del Dataset:")
print(f"Número de clases: {dataset_info['num_classes']}")
print(f"Clases: {dataset_info['class_names']}")
print(f"Total de muestras: {dataset_info['total_samples']}")
print(f"Resolución de muestra: {dataset_info['sample_resolution']}")
print("\nMuestras por clase:")
for class_name, count in dataset_info['samples_per_class'].items():
    print(f"  {class_name}: {count}")

# Crear data loaders
train_loader, val_loader, test_loader, class_names = create_data_loaders(
    data_dir, batch_size=batch_size, input_size=input_size, seed=42
)

nChannels = 3
nClasses = len(class_names)
print(f"\nCanales de entrada: {nChannels}")
print(f"Número de clases: {nClasses}")

## 2. LeNet Base - Entrenamiento

In [ ]:
# Crear modelo CustomLenet base (con BN y Dropout)
lenet_base = CustomLenet(nChannels, nClasses).to(device)
params_base = count_parameters(lenet_base)
print(f"CustomLenet Base - Parámetros: {params_base:,} ({params_base/1e6:.2f}M)")

# Entrenar modelo base
save_path = '../checkpoints/customlenet_base_best.pth'
os.makedirs('../checkpoints', exist_ok=True)

print("\nEntrenando CustomLenet Base...")
history_base, avg_time_base = train_model(
    lenet_base, train_loader, val_loader, 
    epochs=60, optimizer_name='SGD', lr=0.001, 
    device=device, save_path=save_path
)

print(f"Tiempo promedio por época: {avg_time_base:.2f}s")

In [ ]:
# Evaluar modelo base en test
lenet_base.load_state_dict(torch.load(save_path))
test_acc_base, test_f1_base, conf_matrix_base, report_base = test_model(
    lenet_base, test_loader, device, class_names
)

print(f"CustomLenet Base - Test Accuracy: {test_acc_base:.2f}%")
print(f"CustomLenet Base - Test F1: {test_f1_base:.2f}%")
print("\nReporte de clasificación:")
print(report_base)

## 3. CustomLenet sin BatchNorm - Variante 1

In [ ]:
# Crear modelo CustomLenet sin BatchNorm
lenet_nobn = CustomLenet_NoBN(nChannels, nClasses).to(device)
params_nobn = count_parameters(lenet_nobn)
print(f"CustomLenet sin BN - Parámetros: {params_nobn:,} ({params_nobn/1e6:.2f}M)")

# Entrenar modelo sin BatchNorm
save_path_nobn = '../checkpoints/customlenet_nobn_best.pth'

print("\nEntrenando CustomLenet sin BatchNorm...")
history_nobn, avg_time_nobn = train_model(
    lenet_nobn, train_loader, val_loader, 
    epochs=60, optimizer_name='SGD', lr=0.001, 
    device=device, save_path=save_path_nobn
)

print(f"Tiempo promedio por época: {avg_time_nobn:.2f}s")

In [ ]:
# Evaluar modelo sin BatchNorm en test
lenet_nobn.load_state_dict(torch.load(save_path_nobn))
test_acc_nobn, test_f1_nobn, conf_matrix_nobn, report_nobn = test_model(
    lenet_nobn, test_loader, device, class_names
)

print(f"CustomLenet sin BN - Test Accuracy: {test_acc_nobn:.2f}%")
print(f"CustomLenet sin BN - Test F1: {test_f1_nobn:.2f}%")
print(f"Diferencia en Accuracy vs Base: {test_acc_nobn - test_acc_base:.2f}%")
print(f"Diferencia en F1 vs Base: {test_f1_nobn - test_f1_base:.2f}%")

## 4. CustomLenet sin Dropout - Variante 2

In [ ]:
# Crear modelo CustomLenet sin Dropout
lenet_nodropout = CustomLenet_NoDropout(nChannels, nClasses).to(device)
params_nodropout = count_parameters(lenet_nodropout)
print(f"CustomLenet sin Dropout - Parámetros: {params_nodropout:,} ({params_nodropout/1e6:.2f}M)")

# Entrenar modelo sin Dropout
save_path_nodropout = '../checkpoints/customlenet_nodropout_best.pth'

print("\nEntrenando CustomLenet sin Dropout...")
history_nodropout, avg_time_nodropout = train_model(
    lenet_nodropout, train_loader, val_loader, 
    epochs=60, optimizer_name='SGD', lr=0.001, 
    device=device, save_path=save_path_nodropout
)

print(f"Tiempo promedio por época: {avg_time_nodropout:.2f}s")

In [ ]:
# Evaluar modelo sin Dropout en test
lenet_nodropout.load_state_dict(torch.load(save_path_nodropout))
test_acc_nodropout, test_f1_nodropout, conf_matrix_nodropout, report_nodropout = test_model(
    lenet_nodropout, test_loader, device, class_names
)

print(f"CustomLenet sin Dropout - Test Accuracy: {test_acc_nodropout:.2f}%")
print(f"CustomLenet sin Dropout - Test F1: {test_f1_nodropout:.2f}%")
print(f"Diferencia en Accuracy vs Base: {test_acc_nodropout - test_acc_base:.2f}%")
print(f"Diferencia en F1 vs Base: {test_f1_nodropout - test_f1_base:.2f}%")

## 5. Visualización de Resultados

In [ ]:
# Crear directorio de resultados
os.makedirs('../results', exist_ok=True)

# Visualizar curvas de entrenamiento para cada modelo
plot_training_curves(history_base, "CustomLenet Base", '../results/customlenet_base_curves.png')
plot_training_curves(history_nobn, "CustomLenet sin BN", '../results/customlenet_nobn_curves.png')
plot_training_curves(history_nodropout, "CustomLenet sin Dropout", '../results/customlenet_nodropout_curves.png')

In [ ]:
# Matrices de confusión
plot_confusion_matrix(conf_matrix_base, class_names, "CustomLenet Base", '../results/customlenet_base_confusion.png')
plot_confusion_matrix(conf_matrix_nobn, class_names, "CustomLenet sin BN", '../results/customlenet_nobn_confusion.png')
plot_confusion_matrix(conf_matrix_nodropout, class_names, "CustomLenet sin Dropout", '../results/customlenet_nodropout_confusion.png')

In [ ]:
# Resumen de resultados CustomLenet
results_lenet = {
    'Modelo': ['CustomLenet Base', 'CustomLenet sin BN', 'CustomLenet sin Dropout'],
    'Params (M)': [params_base/1e6, params_nobn/1e6, params_nodropout/1e6],
    't/epoca (s)': [avg_time_base, avg_time_nobn, avg_time_nodropout],
    'Val Acc': [max(history_base['val_acc']), max(history_nobn['val_acc']), max(history_nodropout['val_acc'])],
    'Val F1': [max(history_base['val_f1']), max(history_nobn['val_f1']), max(history_nodropout['val_f1'])],
    'Test Acc': [test_acc_base, test_acc_nobn, test_acc_nodropout],
    'Test F1': [test_f1_base, test_f1_nobn, test_f1_nodropout]
}

import pandas as pd
df_lenet = pd.DataFrame(results_lenet)
print("Resultados CustomLenet y Variantes:")
print(df_lenet.round(2))

# Guardar resultados
df_lenet.to_csv('../results/customlenet_results.csv', index=False)

# Guardar historiales para análisis posterior
import pickle
with open('../results/customlenet_histories.pkl', 'wb') as f:
    pickle.dump({
        'base': history_base,
        'nobn': history_nobn,
        'nodropout': history_nodropout
    }, f)